In [1]:
# imports
import pandas as pd
import StringIO
import zipfile
import requests
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import re
import json

In [11]:
# read in capital bike share and citi bike data frames
df_capital = pd.read_csv('C:/Users/Michael/projects/capitalbikeshare/Q22015CapitalBikeShare.csv',index_col = 0)
df_citi = pd.read_csv('C:/Users/Michael/projects/capitalbikeshare/June2015CitiBike.csv', index_col = 0)

In [12]:
# let's take a look at what we're working with
print 'df_citi summary','\n'
print df_citi.shape, '\n'
print df_citi.dtypes, '\n'
print df_citi.columns.values,'\n'
print df_citi.head()

print '\n', '\n'

print 'df_capital summary','\n'
print df_capital.shape, '\n'
print df_capital.dtypes, '\n'
print df_capital.columns.values,'\n'
print df_capital.head()

df_citi summary 

(915072, 30) 

tripduration                 int64
starttime                   object
stoptime                    object
start station id             int64
start station name          object
start station latitude     float64
start station longitude    float64
end station id               int64
end station name            object
end station latitude       float64
end station longitude      float64
bikeid                       int64
usertype                    object
birth year                 float64
gender                       int64
altitude                   float64
availableBikes             float64
availableDocks             float64
city                       float64
id                         float64
landMark                   float64
lastCommunicationTime       object
latitude                   float64
longitude                  float64
postalCode                 float64
stAddress1                  object
stAddress2                 float64
statusKey             

In [13]:
# let's align the datasets
# include only june data from the capital bikeshare dataset
df_cap = df_capital[df_capital.Start_month == 6].reset_index()

# add duration in seconds to capital bikeshare
df_cap['Duration(seconds)'] = df_cap['Duration(ms)'].divide(1000)
df_cap['Duration(seconds)'] = df_cap['Duration(seconds)'].round(decimals=0)
df_citi['Duration(seconds)'] = df_citi['tripduration']

# reduce the datasets to only show columns in common with each other
df_citi = df_citi[['tripduration','Duration(seconds)','starttime','start station name','stoptime','end station name','usertype','bikeid','availableBikes','availableDocks','totalDocks','testStation']]
df_cap = df_cap[['Duration(h:m:s)','Duration(seconds)','Start_date','Start_station','End_date','End_station','Subscription_type','Bike_number','nbbikes','nbemptydocks','totaldocks','temporary']]

In [14]:
# how many test stations are there?
print df_citi['testStation'].value_counts()
print df_cap['temporary'].value_counts()
# there aren't any test stations in the dataframe

False    915072
dtype: int64
False    311692
dtype: int64


In [15]:
# convert tripduration in citi to date type
df_citi['tripduration'] = pd.to_timedelta(df_citi['tripduration'],unit='s')

# re-name the columns so that they are the same in both dataframes
df_citi.columns = df_cap.columns

# add an source column
df_citi['datasource'] = 'citibike'
df_cap['datasource'] = 'capitalbikeshare'

# combine dataframes
df_bikes = df_citi.append(df_cap, ignore_index=True)
df_bikes.head()

,Duration(h:m:s),Duration(seconds),Start_date,Start_station,End_date,End_station,Subscription_type,Bike_number,nbbikes,nbemptydocks,totaldocks,temporary,datasource
0,0:22:18,1338,6/1/2015 0:00,MacDougal St & Prince St,6/1/2015 0:22,W 45 St & 8 Ave,Subscriber,20721,26,4,30,False,citibike
1,0:04:50,290,6/1/2015 0:00,St Marks Pl & 1 Ave,6/1/2015 0:05,Allen St & E Houston St,Subscriber,21606,1,26,27,False,citibike
2,0:10:34,634,6/1/2015 0:01,Greenwich Ave & Charles St,6/1/2015 0:11,W 26 St & 10 Ave,Subscriber,16595,10,28,39,False,citibike
3,0:02:39,159,6/1/2015 0:01,Allen St & Hester St,6/1/2015 0:04,Forsyth St & Broome St,Subscriber,16949,2,41,43,False,citibike
4,0:20:33,1233,6/1/2015 0:02,University Pl & E 14 St,6/1/2015 0:22,S 5 Pl & S 4 St,Customer,17028,23,11,36,False,citibike


In [18]:
# add day of day & hour start to data
dates = pd.DatetimeIndex(df_bikes.Start_date)
df_bikes['Start_day'] = dates.day
df_bikes['Start_hour'] = dates.hour
df_bikes.head()

,Duration(h:m:s),Duration(seconds),Start_date,Start_station,End_date,End_station,Subscription_type,Bike_number,nbbikes,nbemptydocks,totaldocks,temporary,datasource,Start_day,Start_hour
0,0:22:18,1338,6/1/2015 0:00,MacDougal St & Prince St,6/1/2015 0:22,W 45 St & 8 Ave,Subscriber,20721,26,4,30,False,citibike,1,0
1,0:04:50,290,6/1/2015 0:00,St Marks Pl & 1 Ave,6/1/2015 0:05,Allen St & E Houston St,Subscriber,21606,1,26,27,False,citibike,1,0
2,0:10:34,634,6/1/2015 0:01,Greenwich Ave & Charles St,6/1/2015 0:11,W 26 St & 10 Ave,Subscriber,16595,10,28,39,False,citibike,1,0
3,0:02:39,159,6/1/2015 0:01,Allen St & Hester St,6/1/2015 0:04,Forsyth St & Broome St,Subscriber,16949,2,41,43,False,citibike,1,0
4,0:20:33,1233,6/1/2015 0:02,University Pl & E 14 St,6/1/2015 0:22,S 5 Pl & S 4 St,Customer,17028,23,11,36,False,citibike,1,0


In [20]:
# flip into CSV file
df_bikes.to_csv('C:/Users/Michael/projects/capitalbikeshare/CombinedBikeData.csv')